In [1]:
import __casac__ as cc
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.io import fits as pf
from dsacalib.utils import *
from dsacalib.calib import * 
from dsacalib.plotting import *
from dsacalib.fringestopping import *
from astropy.time import Time
%matplotlib inline

In [2]:
# Define the file, source and pointing parameters
datadir= '/home/simard/dsa_calib/data'
fl     = '{0}/J1459+7140/flagged.fits'.format(datadir) 
cal    = src('J1459+7140','14h59m07.63s','+71d40m19.9s',7.47)
sr0    = src('J1520+7225','15h20m47.46s','+72d25m04.80s',1.5491)
antenna_order = [8,5,7,4,3,0,9,1,6,2]
nant   = len(antenna_order)

## Fringestop on Zenith and Integrate

In [3]:
fobs, blen, bname, tstart, tstop, vis, mjd, transit_idx = \
    read_psrfits_file(fl,cal,dur=50*u.min,
                      antpos='{0}/antpos_ITRF.txt'
                      .format(datadir))

File covers 1.01 h hours from MJD 58753.925137845064 to 58753.96714612979


In [4]:
generate_fringestopping_table(blen,nint=25)

(45, 25)
(45, 25)


In [4]:
vis = fringestop_on_zenith(vis,fobs,nint=25,
        fstable='/home/simard/dsa_calib/fringestopping_table.npz')

AssertionError: 

In [ ]:
zenith_ra=Time(np.mean([tstart,tstop]),format='mjd').sidereal_time('apparent',
                                    longitude=ct.ovro_lon*u.rad).to_string()

In [ ]:
zenith = src('zenith',zenith_ra,ct.pt_dec,1.)

In [ ]:
msname = Time(tstart,format='mjd').isot
convert_to_ms(zenith,vis,tstart,'{0}.ms'.format(msname),bname,
              tsamp=ct.tsamp*25,nint=1,
              antpos='{0}/antpos_ITRF.txt'.format(datadir))

In [ ]:
nt   = vis.shape[1]
tobs = tstart + np.arange(0.5,nt+0.5)*ct.tsamp*25/ct.seconds_per_day
divide_visibility_sky_model(vis,blen,[cal],tobs,fobs)

In [ ]:
convert_to_ms(cal,vis,tstart,'{0}.ms'.format(cal.name),bname,
              tsamp=ct.tsamp*25,nint=1,
              antpos='{0}/antpos_ITRF.txt'.format(datadir))

## Delay calibration

In [ ]:
delay_calibration(cal.name,cal.name)

#### Use deviations from average delay calibration to flag bad times

In [ ]:
bad_times,times = get_bad_times(cal.name,cal.name,nant)
flag_badtimes(cal.name,times,bad_times,nant)

In [ ]:
times, antenna_delays, kcorr = plot_antenna_delays(cal.name,cal.name,antenna_order)

#### Flag antennas that we know are bad and short baselines with crosstalk

In [ ]:
flag_antenna(cal.name,4)
flag_antenna(cal.name,2)
flag_antenna(cal.name,"3,5,7,9&")

#### Gain calibration

In [ ]:
gain_calibration(cal.name,cal.name)

#### Look at solutions

In [ ]:
plot_gain_calibration(cal.name,cal.name,antenna_order)

In [ ]:
plot_image(cal.name,'corrected',cal,verbose=True)

## Apply solution to target source

In [ ]:
vis,vist = extract_vis_from_ms(msname)

In [ ]:
divide_visibility_sky_model(vis,blen,[sr0],tobs,fobs,phase_only=True)

In [ ]:
convert_to_ms(sr0,vis,tstart,'{0}.ms'.format(sr0.name),bname,
              tsamp=ct.tsamp*25,nint=1,
              antpos='{0}/antpos_ITRF.txt'.format(datadir))

#### Flag bad antennas and short baselines

In [ ]:
flag_antenna(sr0.name,4)
flag_antenna(sr0.name,2)
flag_antenna(sr0.name,"3,5,7,9&")

#### Calibrate

In [ ]:
apply_calibration(sr0.name,cal.name,msnamecal=cal.name)

#### Look at calibrated data

In [ ]:
ms = cc.ms.ms()
ms.open('{0}.ms'.format(sr0.name))
data = ms.getdata(['corrected_data'])
vis_cal = data['corrected_data'].reshape(2,625,-1,45).T
ms.close()

In [ ]:
plot_vis_freq(vis_cal,fobs,bname)

In [ ]:
plot_vis_time(vis_cal,tobs,bname)

In [ ]:
plot_image('{0}'.format(sr0.name),'corrected',sr0,verbose=True,
          npix=128)

In [ ]:
plot_image('{0}'.format(sr0.name),'corrected',sr0,verbose=True)